In [3]:
import geopandas as gpd
import geemap
from outils.initializeGEE import initializeEarthEngine

# Autentica e inicializa o Google Earth Engine
initializeEarthEngine()

True

In [6]:
gdf = gpd.read_parquet('data\\terrasIndigenasBr.parquet')
gdf_ee = geemap.geopandas_to_ee(gdf)

In [ ]:
gdf_ee